In [46]:
import pandas as pd
import os
import numpy as np
import re

mypath="data/temperatures/"

df=pd.DataFrame()

for filename in os.listdir(mypath):
    print(filename)
    df_temp = pd.read_csv(mypath+filename,compression='gzip', sep = ';')
    df_temp.rename(columns={df_temp.columns[0]:'numer_sta'},inplace=True)
    #keep only relevant columns
    df_temp=df_temp[['numer_sta','t','date']]
    #Convert temperature from Kelvin to Celsius degrees :
    df_temp.loc[df_temp.t == 'mq','t'] = np.nan
    df_temp.t = pd.to_numeric(df_temp.t)- 273.15
    #drop date values that are not in the right format and convert to date format
    df_temp['date']=df_temp['date'].apply(
        lambda x: 
        x if re.match('[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]',str(x)) else np.nan)
    df_temp.date = pd.to_datetime(df_temp.date, format='%Y%m%d%H%M%S')
    #keep only stations in France Metropolitaine
    list_outremer_corse = [71805, 81408, 81415, 81405, 81401, 78894, 78890, 78897, 78922, 78925, 89642, 
                 61998, 61996, 61997, 61980, 61976, 61968, 67005, 61970, 61972,7790,7761]
    df_temp=df_temp[df_temp.numer_sta.isin(list_outremer_corse)==False]
    #aggregate temperatures by date, apply mean
    grouped_temp =  df_temp.groupby('date').t.mean()
    df_temp = pd.DataFrame(grouped_temp).reset_index()
    #Add a column giving the hour
    df_temp['hour'] = df_temp.date.apply(lambda u: u.strftime('%H:%M'))
    df_temp['date'] = df_temp.date.apply(lambda u: u.date())
    df=pd.concat([df,df_temp])
    #Pivot to get the same dataframe shape as electricity
    #df_temp.pivot(index='date',columns='hour', values='t')
    #df_temp['date'] = df_temp.index
    #df_temp = df_temp.reset_index(drop=True)

df.head()

synop.201609.csv.gz.cpgz
synop.201605.csv.gz.cpgz
synop.201610.csv.gz.cpgz
synop.201606.csv.gz.cpgz
synop.201603.csv.gz.cpgz
synop.201611.csv.gz.cpgz
synop.201604.csv.gz.cpgz
synop.201601.csv.gz.cpgz
synop.201608.csv.gz.cpgz
synop.201602.csv.gz.cpgz
synop.201607.csv.gz.cpgz
synop.201612.csv.gz.cpgz


,date,t,hour
0,2016-09-01,18.943103,00:00
1,2016-09-01,17.033333,03:00
2,2016-09-01,18.136364,06:00
3,2016-09-01,21.832759,09:00
4,2016-09-01,25.161404,12:00


In [48]:
df.to_csv('data/temp_2016.csv',sep=',')